In [1]:
# --- 환경 설정 및 라이브러리 설치 ---
from IPython.display import clear_output

# 시스템 라이브러리 (음성 속도 조절용)
!apt-get update && apt-get install -y ffmpeg

# Python 라이브러리
!pip install -q gradio google-generativeai gtts scikit-learn pandas

clear_output()
print("✅ 모든 라이브러리 설치가 완료되었습니다.")

✅ 모든 라이브러리 설치가 완료되었습니다.


In [14]:
# -*- coding: utf-8 -*-

# --- 필수 라이브러리 임포트 ---
import gradio as gr
import asyncio
import google.generativeai as genai
from gtts import gTTS
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re
import subprocess
from google.colab import userdata
from google.api_core import exceptions as google_exceptions

# --- 1. RAG 검색기 클래스 정의 ---
class RAGChatBot:
    def __init__(self, filepath):
        try:
            # CSV 파일 로드 및 벡터화
            self.data = pd.read_csv(filepath)
            # 중요: CSV 파일의 실제 열 이름으로 수정
            #self.questions = self.data['질문 (Question)'].tolist()
            #self.answers = self.data['해설 (Answer)'].tolist()
            self.questions = self.data['Q'].tolist()
            self.answers = self.data['A'].tolist()

            self.vectorizer = TfidfVectorizer()
            self.question_vectors = self.vectorizer.fit_transform(self.questions)
            print("✅ RAG 챗봇 초기화 완료: CSV 데이터 로드 및 벡터화 성공")
        except FileNotFoundError:
            print(f"❌ 오류: '{filepath}' 파일을 찾을 수 없습니다. 파일이 업로드되었는지 확인하세요.")
            raise
        except KeyError as e:
            print(f"❌ 오류: CSV 파일에서 '{e}' 열을 찾을 수 없습니다. 열 이름을 확인하세요.")
            print("   (예: '질문 (Question)', '해설 (Answer)')")
            raise

    def retrieve_context(self, user_query):
        """사용자 질문과 가장 유사한 답변(Context)을 찾는 함수"""
        if not user_query:
            return None
        input_vector = self.vectorizer.transform([user_query])
        similarities = cosine_similarity(input_vector, self.question_vectors)
        best_match_index = similarities.argmax()
        return self.answers[best_match_index]

# --- 2. Gemini 및 TTS 관련 함수 정의 ---

# 프롬프트 규칙
PROMPT_GUIDELINE = """
항상 다음 규칙을 준수하여 질문에 답해 주세요.

- 전문가다운 문체의 존댓말을 사용하고 이모지는 제외합니다.
- 제공된 '참고 자료'가 있다면, 그 내용을 바탕으로 답변을 구성해야 합니다.
- 정답이나 결론을 먼저 제시하고 이후에 설명은 3줄 내외로 간결하게 합니다.
- 설명은 반복없이 요점만 말하고 모르는 경우는 명확히 "모른다"고 답합니다.
- 한글 띄어쓰기를 철저히 지킵니다.
- 강조 표현은 굵은 글씨체만 사용하며 **강조할 말**, "강조할 말" 형식은 사용 금지합니다.
"""

# Gemini 응답 생성 함수
async def get_gemini_response(prompt, model_name="gemini-1.5-flash"):
    try:
        model = genai.GenerativeModel(model_name)
        response = await asyncio.to_thread(model.generate_content, prompt)
        return response.text.strip()
    except google_exceptions.GoogleAPIError as e:
        # 모델 이름 오류 등 API 관련 문제 처리
        if "model not found" in str(e).lower() or "permission" in str(e).lower():
             return f"API 오류: 잘못된 모델 이름('{model_name}')을 사용했거나 권한이 없습니다. 'gemini-1.5-flash' 또는 'gemini-1.5-pro'를 사용해보세요."
        return f"API 오류: {str(e)}"
    except Exception as e:
        return f"기타 오류: {str(e)}"

# 강조마크 제거 및 TTS 처리
def clean_markdown(text):
    text = text.replace("**", "").replace("__", "").replace("~~", "")
    text = re.sub(r"\[(.+?)\]\(.*?\)", r"\1", text)
    return text

def speak_text(text, lang='ko'):
    try:
        if not text: return None
        text = clean_markdown(text)
        filename, fast_filename = "response.mp3", "response_fast.mp3"
        tts = gTTS(text=text, lang=lang)
        tts.save(filename)
        subprocess.run(
            ["ffmpeg", "-y", "-i", filename, "-filter:a", "atempo=1.6", "-vn", fast_filename],
            check=True, capture_output=True, text=True
        )
        return fast_filename
    except Exception as e:
        print(f"음성 출력 오류: {e}")
        return None

# --- 3. 챗봇 핵심 로직 (RAG 파이프라인 결합) ---
use_japanese = False
rag_chatbot_instance = None # RAG 챗봇 인스턴스를 저장할 전역 변수

async def rag_chat_with_gemini(user_input):
    global use_japanese, rag_chatbot_instance

    if rag_chatbot_instance is None:
        return "오류: 챗봇 데이터가 로드되지 않았습니다. 이전 단계에서 CSV 파일을 업로드했는지 확인하세요.", None

    if not user_input.strip():
        return "질문을 입력해 주세요.", None

    # 언어 전환 명령 처리 (RAG 불필요)
    if user_input.strip() in ["일본어로 말해줘", "일본어로 답변해 주세요"]:
        use_japanese = True
        return "앞으로 일본어로 응답하겠습니다。", speak_text("これから日本語で答えます。", lang='ja')
    elif user_input.strip() in ["한국어로 말해줘", "한국어로 답변해 주세요"]:
        use_japanese = False
        return "앞으로 한국어로 응답하겠습니다.", speak_text("앞으로 한국어로 응답하겠습니다.", lang='ko')

    # 1. Retrieval: CSV에서 참고 자료 추출
    retrieved_context = rag_chatbot_instance.retrieve_context(user_input)

    # 2. Augmentation: RAG용 프롬프트 구성
    lang = 'ja' if use_japanese else 'ko'
    if lang == 'ja':
        prompt_template = f"{PROMPT_GUIDELINE}\n\n---参考資料---\n{retrieved_context}\n\n上記の参考資料を基に、次の質問に日本語で答えてください。\n質問: {user_input}"
    else:
        prompt_template = f"{PROMPT_GUIDELINE}\n\n---참고 자료---\n{retrieved_context}\n\n위 참고 자료를 바탕으로 다음 질문에 대해 답변해 주세요.\n질문: {user_input}"

    # 3. Generation: Gemini를 통해 최종 답변 생성
    final_response = await get_gemini_response(prompt_template)

    # 음성 생성 (15자 이상일 때만)
    audio_file = speak_text(final_response, lang=lang) if len(final_response) >= 15 else None

    return final_response, audio_file

# --- 4. Gradio UI 정의 ---
def create_gradio_ui():
    with gr.Blocks(css=".gradio-container { max-width: 400px; margin: auto !important; }") as demo:
        gr.Markdown("## RAG 음성챗봇 MK")
        input_box = gr.Textbox(label="질문 입력", lines=2, placeholder="예: 저장탱크 기초 재사용이 가능한가요?")
        submit_btn = gr.Button("전송", size="sm")
        output_text = gr.Textbox(label="응답", lines=5, interactive=False)
        audio_output = gr.Audio(label="음성응답", type="filepath", autoplay=True)

        submit_btn.click(fn=rag_chat_with_gemini, inputs=input_box, outputs=[output_text, audio_output])

    return demo

print("✅ 함수 및 클래스 정의가 완료되었습니다.")

✅ 함수 및 클래스 정의가 완료되었습니다.


In [15]:
# --- API 키 설정 (Colab 보안 비밀 사용 권장) ---
try:
    # 1. Colab의 '보안 비밀' 기능에 'GEMINI_API_KEY'라는 이름으로 키를 저장하세요.
    GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')
    genai.configure(api_key=GEMINI_API_KEY)
    print("✅ Gemini API 키가 성공적으로 설정되었습니다.")
except Exception as e:
    print("❌ API 키 설정 실패. Colab의 '보안 비밀'에 GEMINI_API_KEY가 올바르게 설정되었는지 확인하세요.")
    print("   (오류: ", e, ")")


# --- CSV 데이터 파일 업로드 및 챗봇 인스턴스 생성 ---
from google.colab import files

print("\n--- RAG에 사용할 질의응답 CSV 파일을 업로드하세요 ---")
# 이전에 생성한 '가스관계법령_질의해설집.csv' 파일을 업로드합니다.
uploaded = files.upload()

if uploaded:
    filepath = next(iter(uploaded))
    try:
        # 업로드된 파일을 사용하여 RAG 챗봇 인스턴스 생성
        rag_chatbot_instance = RAGChatBot(filepath)
    except Exception as e:
        print(f"챗봇 초기화 중 오류 발생. 프로그램을 중단합니다.")
else:
    print("\n파일이 업로드되지 않았습니다. 챗봇을 실행할 수 없습니다.")

✅ Gemini API 키가 성공적으로 설정되었습니다.

--- RAG에 사용할 질의응답 CSV 파일을 업로드하세요 ---


Saving ChatbotData.csv to ChatbotData (4).csv
✅ RAG 챗봇 초기화 완료: CSV 데이터 로드 및 벡터화 성공


In [16]:
if rag_chatbot_instance:
    print("\n🚀 Gradio UI를 실행합니다...")
    demo = create_gradio_ui()
    demo.launch(debug=True)
else:
    print("\n챗봇이 초기화되지 않아 UI를 실행할 수 없습니다. 이전 단계를 확인해주세요.")


🚀 Gradio UI를 실행합니다...
It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://71334f632453c67dcd.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://71334f632453c67dcd.gradio.live
